In [2]:
! pip install transformers

In [3]:
! pip install sentencepiece

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import pandas as pd
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader,Dataset

device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
from transformers import BertTokenizer,BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=6)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [8]:
print(model.parameters)

<bound method Module.parameters of BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (Layer

In [9]:
from datasets import load_dataset

# dataset = load_dataset("SetFit/emotion")
# from datasets import load_dataset

dataset = load_dataset("dair-ai/emotion")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset emotion downloaded and prepared to /root/.cache/huggingface/datasets/dair-ai___emotion/split/1.0.0/cca5efe2dfeb58c1d098e0f9eeb200e9927d889b5a03c67097275dfb5fe463bd. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
inputs = tokenizer(['hello world'],padding="max_length",truncation=True,return_tensors="pt",max_length=128)
inputs

{'input_ids': tensor([[ 101, 7592, 2088,  102,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0,

In [12]:
outputs = model(inputs['input_ids'])
outputs

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


SequenceClassifierOutput(loss=None, logits=tensor([[ 0.0434, -0.3377,  0.1346, -0.4797, -0.0717, -0.4622]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [13]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [14]:
def process_data(sample,padding="max_length"):
#     inputs = [item for item in sample['text']]
    
    X = tokenizer(text_target=sample['text'],padding=padding,max_length=128,truncation=True)
    
    X["labels"] = sample["label"]

    return X

In [11]:
# # dataset['train'][:1000]
# dataset['train'] = dataset['train'][:2000]
# dataset['test'] = dataset['test'][:1000]

In [15]:
train_dataset = dataset['train'].map(process_data,batched=True,remove_columns=['text','label'])
test_dataset = dataset['test'].map(process_data,batched=True,remove_columns=['text','label'])


  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [16]:
torch.tensor(train_dataset['input_ids'][0])

tensor([  101,  1045,  2134,  2102,  2514, 26608,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [17]:
model(torch.tensor([train_dataset['input_ids'][0]]))

SequenceClassifierOutput(loss=None, logits=tensor([[-0.0045, -0.2823,  0.1871, -0.4061, -0.0765, -0.4042]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [18]:
train_dataset_d = train_dataset.select(range(100))
test_dataset_d = test_dataset.select(range(100))

train_dataset_d

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 100
})

In [20]:
class BertClassifier(nn.Module):
    def __init__(self,model,num_classes):
        super(BertClassifier,self).__init__()
        self.model = model
        
    def forward(self,x):
        x = self.model(x)
        
        return x

In [21]:
model_0 = BertClassifier(model,6)
model_0.to(device)
print(model_0.parameters)

<bound method Module.parameters of BertClassifier(
  (model): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear

In [18]:
def accuracy(y_pred,y_true):
    return sum((y_pred==y_true))/len(y_true)

In [46]:
model_0(torch.tensor([train_dataset['input_ids'][3]]).to(device))['logits']

tensor([[  6.0232, -13.2564, -13.6209, -13.1209, -12.4737, -12.7266]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [41]:
class EmotionData(Dataset):
    def __init__(self,data):
        self.data = data
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
        input_ids = data['input_ids'][idx]
        attention_mask = data['attention_mask'][idx]
        labels = data['labels'][idx]
        
        return {'input_ids':torch.tensor(input_ids),'attention_mask':torch.tensor(attention_mask),'labels':torch.tensor(labels)}

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 100
})

In [37]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_0.parameters(),lr=0.01)
for i in range(2):
    model_0.train()
    Y_pred = model_0(torch.as_tensor([train_dataset['input_ids'][i]]).to(device))
    print(Y_pred)
    loss = loss_fn(Y_pred['logits'],torch.as_tensor([train_dataset['labels'][i]]).to(device))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
print(loss)
#     print(accuracy(Y_pred,F.one_hot(torch.as_tensor(train_dataset['labels'][i])).to(device)))

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.4238,  0.1265, -0.1755,  0.6909,  0.1299, -0.9170]],
       device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=None, logits=tensor([[-0.2364,  0.6878,  0.0766, -0.1047,  0.4328,  0.0930]],
       device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor(2.2376, device='cuda:0', grad_fn=<NllLossBackward0>)
SequenceClassifierOutput(loss=None, logits=tensor([[-12.6143,   5.4391,   4.9107,   5.8035,   5.1901,   4.1955]],
       device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=None, logits=tensor([[-7.9504,  0.5234,  0.0311,  0.7315, -0.4870, -1.3021]],
       device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor(9.6881, device='cuda:0', grad_fn=<NllLossBackward0>)
SequenceClassifierOutput(loss=None, logits=tensor([[-3.4945, -3.8889, -4.4120, -3.9167, -4

In [21]:
torch.squeeze(F.one_hot(torch.as_tensor(train_dataset['labels'][4]),num_classes=6).to(device))

tensor([0, 0, 0, 1, 0, 0], device='cuda:0')

In [22]:
Y_pred = model_0(torch.as_tensor([train_dataset['input_ids'][3]]).to(device))
loss  = loss_fn(Y_pred,torch.as_tensor([train_dataset['labels'][3]]).to(device))
print(torch.squeeze(Y_pred))

tensor([-14.8620,   6.5938, -12.7250, -13.9139, -14.3188, -13.8228],
       device='cuda:0', grad_fn=<SqueezeBackward0>)


In [23]:
target = torch.randint(0, 10, (4,))
target

tensor([0, 2, 7, 8])

In [47]:
def ModelFit(model,train_dataset,test_dataset,epochs=3):
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
    lr_schedule = ReduceLROnPlateau(optimizer,'min',factor=0.1,patience=2)

    for epoch in range(epochs):
        for i in range(len(train_dataset)):
            model.train()
            Y_pred = model(torch.tensor([train_dataset['input_ids'][i]]).to(device))
            loss  = loss_fn(Y_pred['logits'],torch.tensor([train_dataset['labels'][i]]).to(device))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        for j in range(len(test_dataset)):
            with torch.no_grad():
                model.eval()
                Y_pred_test = model(torch.tensor([test_dataset['input_ids'][j]]).to(device))
                loss_test = loss_fn(Y_pred_test['logits'],torch.tensor([test_dataset['labels'][j]]).to(device))
        
        
        print("Train Loss:",loss,"Test Loss:",loss_test)
        lr_schedule.step(loss_test)
        if loss < 1 and loss_test < 1:
            break
            
    return model

In [48]:
# model_1 = ModelFit(model_0,train_dataset_d,test_dataset_d,epochs=5)

Train Loss: tensor(3.6741, device='cuda:0', grad_fn=<NllLossBackward0>) Test Loss: tensor(2.0280, device='cuda:0')
Train Loss: tensor(6.0211, device='cuda:0', grad_fn=<NllLossBackward0>) Test Loss: tensor(0.3866, device='cuda:0')
Train Loss: tensor(2.4821, device='cuda:0', grad_fn=<NllLossBackward0>) Test Loss: tensor(3.4167, device='cuda:0')
Train Loss: tensor(0.8295, device='cuda:0', grad_fn=<NllLossBackward0>) Test Loss: tensor(2.2572, device='cuda:0')
Train Loss: tensor(4.2961, device='cuda:0', grad_fn=<NllLossBackward0>) Test Loss: tensor(5.0312, device='cuda:0')


In [29]:
# model_2 = ModelFit(model_1,train_dataset_d,test_dataset_d,epochs=5)

Train Loss: tensor(4.5624, device='cuda:0', grad_fn=<NllLossBackward0>) Test Loss: tensor(0.0013, device='cuda:0')
Train Loss: tensor(0.0027, device='cuda:0', grad_fn=<NllLossBackward0>) Test Loss: tensor(2.4980, device='cuda:0')
Train Loss: tensor(0.3204, device='cuda:0', grad_fn=<NllLossBackward0>) Test Loss: tensor(0.0708, device='cuda:0')


In [6]:
! pip install -q accelerate -U

In [7]:
! pip install -q transformers[torch]

In [22]:
from transformers import Trainer,TrainingArguments

model = model.to("cuda")
# training_args = TrainingArguments(
#     output_dir = './bert-mod',
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     fp16=False, # Overflows with fp16
#     learning_rate=0.01,
#     num_train_epochs=5,
#     # logging & evaluation strategies
# #     logging_strategy="steps",
# #     logging_steps=500,
# #     evaluation_strategy="epoch",
# #     save_strategy="epoch",
# #     save_total_limit=2,
# #     load_best_model_at_end=True,
# #     push_to_hub=False,
# )

trainer = Trainer(
    model=model,
    train_dataset = train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/wandb/sdk/wandb_init.py", line 1166, in init
    wi.setup(kwargs)
  File "/opt/conda/lib/python3.10/site-packages/wandb/sdk/wandb_init.py", line 306, in setup
    wandb_login._login(
  File "/opt/conda/lib/python3.10/site-packages/wandb/sdk/wandb_login.py", line 298, in _login
    wlogin.prompt_api_key()
  File "/opt/conda/lib/python3.10/site-packages/wandb/sdk/wandb_login.py", line 221, in promp

Error: An unexpected error occurred

In [30]:
torch.save(model_2.state_dict(),"model_gpu.pt")

In [31]:
model_2.to("cpu")
torch.save(model_2.state_dict(),"model_cpu.pt")

In [33]:
from IPython.display import FileLink
FileLink(r'model_gpu.pt')

/kaggle/working/model_gpu.pt